In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nowcast-train/train.csv
/kaggle/input/nowcastsensorcsv/sensor.csv


In [2]:
import pandas as pd
df_train = pd.read_csv("../input/nowcast-train/train.csv")

In [3]:
df_train.head()

,chunk_id,rain
0,2397129cde,1
1,b941e4e6a5,0
2,1c5035d5b3,0
3,99247f4b44,1
4,d42968a269,0


In [4]:
df_train.isna().sum()

chunk_id    0
rain        0
dtype: int64

In [5]:
df_train.describe()

,rain
count,144836.000000
mean,0.600748
std,0.828518
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,3.000000


In [6]:
sensor = pd.read_csv("../input/nowcastsensorcsv/sensor.csv")
sensor.head()

,chunk_id,stationid,date,time,tempc,feelslikec,windspeed_kph,windgust_kph,pressure_mb,humidity,visibility_km,cloud,heatindexc,dewpointc,windchillc,uvindex,isday,rain
0,2397129cde,5661,2022-01-03,0,23.6,25.4,8.6,18.0,1004.4,74.3,10.0,89.0,25.4,18.8,23.6,1,0,1
1,2397129cde,5661,2022-01-03,1,23.9,25.7,8.6,18.0,1004.3,73.2,10.0,8.9,25.7,18.8,23.9,1,0,0
2,2397129cde,5661,2022-01-03,2,23.6,25.4,6.1,13.0,1004.1,73.8,10.0,75.0,25.4,18.7,23.6,1,0,1
3,2397129cde,5661,2022-01-03,3,23.1,25.1,4.3,9.0,1004.0,72.2,10.0,8.6,25.1,17.8,23.1,1,0,0
4,2397129cde,5661,2022-01-03,4,23.0,25.0,4.0,8.3,1004.2,70.1,10.0,6.5,25.0,17.3,23.0,1,0,0


In [7]:
sensor.describe()

,stationid,time,tempc,feelslikec,windspeed_kph,windgust_kph,pressure_mb,humidity,visibility_km,cloud,heatindexc,dewpointc,windchillc,uvindex,isday,rain
count,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06,5.551993e+06
mean,1.478272e+07,1.147173e+01,7.469669e+00,5.524368e+00,1.255182e+01,1.890629e+01,1.015673e+03,7.955708e+01,8.131385e+00,6.142525e+01,8.022579e+00,3.634837e+00,5.037958e+00,2.114920e+00,4.149510e-01,6.280348e-01
std,5.128582e+06,6.901471e+00,1.434850e+01,1.713483e+01,9.476962e+00,1.341932e+01,1.047271e+01,1.712193e+01,3.126041e+00,3.560877e+01,1.507877e+01,1.328810e+01,1.648302e+01,1.973910e+00,4.927136e-01,8.475108e-01
min,5.661000e+03,0.000000e+00,-5.330000e+01,-6.590000e+01,0.000000e+00,0.000000e+00,9.502000e+02,3.100000e+00,0.000000e+00,0.000000e+00,-5.330000e+01,-5.330000e+01,-6.590000e+01,1.000000e+00,0.000000e+00,0.000000e+00
25%,1.243522e+07,6.000000e+00,-1.600000e+00,-5.600000e+00,5.400000e+00,9.000000e+00,1.009600e+03,7.030000e+01,7.000000e+00,2.730000e+01,-1.600000e+00,-5.100000e+00,-5.600000e+00,1.000000e+00,0.000000e+00,0.000000e+00
50%,1.537095e+07,1.100000e+01,6.500000e+00,3.800000e+00,1.010000e+01,1.550000e+01,1.014500e+03,8.410000e+01,1.000000e+01,7.200000e+01,6.500000e+00,2.800000e+00,3.800000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,1.870028e+07,1.700000e+01,2.050000e+01,2.050000e+01,1.690000e+01,2.520000e+01,1.022300e+03,9.320000e+01,1.000000e+01,9.860000e+01,2.060000e+01,1.650000e+01,2.050000e+01,2.000000e+00,1.000000e+00,1.000000e+00
max,2.171693e+07,2.300000e+01,4.800000e+01,5.340000e+01,1.170000e+02,1.494000e+02,1.092900e+03,1.000000e+02,1.000000e+01,1.000000e+02,5.340000e+01,2.890000e+01,4.800000e+01,1.100000e+01,1.000000e+00,3.000000e+00


In [8]:
sensor.isna().sum()

chunk_id         0
stationid        0
date             0
time             0
tempc            0
feelslikec       0
windspeed_kph    0
windgust_kph     0
pressure_mb      0
humidity         0
visibility_km    0
cloud            0
heatindexc       0
dewpointc        0
windchillc       0
uvindex          0
isday            0
rain             0
dtype: int64

In [9]:
sensor["chunk_id"] = sensor["chunk_id"].astype("object")

In [10]:
main_cols = sensor.columns.difference(["rain", "chunk_id", "date"]).tolist()
target = "rain"
label = sensor[target]
X = sensor[main_cols]
y = label

# Train - test - Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

In [11]:
# Modelling
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score, make_scorer



In [13]:
pred = model.predict(X_test)

In [14]:
pred

array([ 0.35245243,  0.56308552,  0.29633251, ...,  0.55241685,
       -0.03345987,  1.01152146])

In [15]:
from sklearn.metrics import mean_absolute_error

# Check the MAE score of the model
print(f'LinearRegression MAE score on the X_test is: {mean_absolute_error(y_test, pred)}')

LinearRegression MAE score on the X_test is: 0.5255361916954805


In [16]:
model.score(X, y)

0.3654709760941024

# Pipeline and Cross-validation

In [17]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate # for the generalization performance


model = make_pipeline(LinearRegression())


In [18]:
cv_results = cross_validate(model, X, y)
cv_results

{'fit_time': array([3.04225302, 2.75050235, 2.74774265, 2.74607015, 3.45250702]),
 'score_time': array([0.08396935, 0.08163762, 0.08456135, 0.08180737, 0.08431149]),
 'test_score': array([0.34657878, 0.35804522, 0.34639735, 0.35757559, 0.31341619])}

In [19]:
scores = cv_results["test_score"]
print(f"The accuracy is {scores.mean():.3f} +/- {scores.std():.3f}")

The accuracy is 0.344 +/- 0.016


# Evaluation Using `Quadratic Weighted Kappa`

In [21]:
sample = pd.read_csv("../input/nowcast-sample/sample_submission.csv")
sub = sample.copy()
sub.target = pred
sub.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


,chunk_id,rain_prediction
0,94f8093db4,0
1,bad09bc8bb,0
2,d1ac8d96f4,0
3,12ebf95f1c,0
4,e3674c136b,0


In [23]:
sub.to_csv("Baseline2.csv", index = False)

In [24]:
sub.tail()

,chunk_id,rain_prediction
96550,bbae581394,0
96551,9ddd5ccf90,0
96552,91507a3101,0
96553,8408d5f488,0
96554,87513dda65,0
